# NLP with RoBERTa
### How do you get the fixed representation? Did you do pooling or something?

Yes, pooling is required to get a fixed representation of a sentence. In the default strategy REDUCE_MEAN, I take the second-to-last hidden layer of all of the tokens in the sentence and do average pooling.

In [34]:
import torch
import json
import torch.nn.functional as F

In [5]:
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
roberta.eval()
print()

Using cache found in /home/dev/.cache/torch/hub/pytorch_fairseq_master


loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz from cache at /home/dev/.cache/torch/pytorch_fairseq/83e3a689e28e5e4696ecb0bbb05a77355444a5c8a3437e0f736d8a564e80035e.c687083d14776c1979f3f71654febb42f2bb3d9a94ff7ebdfe1ac6748dba89d2
| dictionary: 50264 types



In [9]:
movies = json.load(open("../../../../data/parsed/omdb.json", "r") )

In [35]:
features = roberta.extract_features(roberta.encode(movies['1']['omdb']['Plot']))

In [51]:
pooled_features = F.avg_pool2d(features, (features.size(1), 1)).squeeze()

In [53]:
pooled_features

tensor([-0.0292,  0.0314, -0.2926,  ..., -0.0074,  0.0038,  0.2130],
       grad_fn=<SqueezeBackward0>)